# P10 - Réalisez une application de recommandation de contenu

In [6]:
# Parameters
metadata_path = "../data/deps/articles_metadata.csv"
output_path = "../data/outs/model/model_svd.pkl"
clicks_path = "../data/deps/clicks_sample.csv"
n_factors = 100  # Exemple de paramètre pour ton modèle SVD

In [4]:
import pandas as pd

metadata = pd.read_csv(metadata_path)

# Affichage du nombre de lignes (donc d'articles)
print("Nombre total d’articles (rows) :", metadata.shape[0])

# Vérification unicité
print("Nombre d’articles uniques (article_id) :", metadata['article_id'].nunique())

# Aperçu rapide
print(metadata.head())

Nombre total d’articles (rows) : 364047
Nombre d’articles uniques (article_id) : 364047
   article_id  category_id  created_at_ts  publisher_id  words_count
0           0            0  1513144419000             0          168
1           1            1  1405341936000             0          189
2           2            1  1408667706000             0          250
3           3            1  1408468313000             0          230
4           4            1  1407071171000             0          162


## Train model SVD

In [7]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
import pickle

# 1. Chargement des données de clics
def load_data(filepath=clicks_path):
    df = pd.read_csv(filepath)
    # On suppose que chaque clic = score implicite de 1
    df["rating"] = 1
    return df[["user_id", "click_article_id", "rating"]]

# 2. Entraînement du modèle SVD
def train_model(df):
    reader = Reader(rating_scale=(0, 1))
    data = Dataset.load_from_df(df, reader)
    trainset = data.build_full_trainset()

    model = SVD()
    model.fit(trainset)

    return model, trainset

# 3. Générer les 5 meilleures recommandations pour un utilisateur donné
def get_top_5(user_id, model, trainset, n=5):
    all_article_ids = trainset._raw2inner_id_items.keys()
    read_articles = set([j for (j, _) in trainset.ur[trainset.to_inner_uid(user_id)]]) \
        if user_id in trainset._raw2inner_id_users else set()

    candidates = [aid for aid in all_article_ids if aid not in read_articles]

    predictions = [
        (aid, model.predict(user_id, aid).est)
        for aid in candidates
    ]
    top_n = sorted(predictions, key=lambda x: x[1], reverse=True)[:n]
    return [int(article_id) for article_id, _ in top_n]

# 4. Exemple d’utilisation
if __name__ == "__main__":
    df = load_data()
    model, trainset = train_model(df)

    # Tester avec un user_id existant (à adapter)
    sample_user_id = df["user_id"].iloc[0]
    recommendations = get_top_5(sample_user_id, model, trainset)
    print(f"Top 5 articles pour l’utilisateur {sample_user_id} : {recommendations}")

    # Sauvegarde du modèle pour usage ultérieur
    with open(output_path, "wb") as f:
        pickle.dump((model, trainset), f)    

Top 5 articles pour l’utilisateur 0 : [68866, 96663, 119592, 168868, 286413]


## Curl

```bash
curl -X POST http://localhost:7071/api/RecommendArticles \
  -H "Content-Type: application/json" \
  -d '{"user_id": 42}'
```